In [1]:
#Install pip packages in the curret Jupyter kernal
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade pyodbc
!{sys.executable} -m pip install polars

  Using cached pip-23.2-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ---------------------------------------- 0.0/69.7 kB ? eta -:--:--
     ---------------------------------------- 69.7/69.7 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: pyodbc
    Found existing installation: pyodbc 4.0.34
    Uninstalling pyodbc-4.0.34:
      Successfully uninstalled pyodbc-4.0.34
  Obtaining dependency information for polars from https://files.pythonhosted.org/packages/c5/18/4f0fbacc2f04eb1ef64cb7fe452d849d32c17bb10543391cfe7ac0cd4f9b/polars-0.18.7-cp38-abi3-win_amd64.whl.metadata
  Using cached polars-0.18.7-cp38-abi3-win_amd64.whl.metadata (14 kB)
Using cached polars-0.18.7-cp38-abi3-win_amd64.whl (19.8 MB)


In [139]:
def pop_random(lst):
    idx = random.randrange(0, len(lst))
    return lst.pop(idx)

wld_list = ['W'] * 45 + ['L'] * 45 + ['D'] * 10

def match_dict(n):
    lst = ["Player " + str(i) for i in range(n)]
    pairs = []
    pair_dict = {}
    point_dict = {}
    while lst:
        rand1 = pop_random(lst)
        rand2 = pop_random(lst)
        pair = [rand1, rand2]
        pairs.append(pair)
    for pair in pairs:
        pair_dict[pair[0]] = pair[1]
        pair_dict[pair[1]] = pair[0]
        wld = random.choice(wld_list)
        if wld == "W":
            point_dict[pair[0]] = 3
            point_dict[pair[1]] = 0
        elif wld == "L":
            point_dict[pair[0]] = 0
            point_dict[pair[1]] = 3
        elif wld == "D":
            point_dict[pair[0]] = 1
            point_dict[pair[1]] = 1
        
    return pair_dict, point_dict

sco_list = ['2-1'] * 29 + ['2-0'] * 15 + ['1-0-1'] * 1 + ['1-2'] * 29 + ['0-2'] * 15 + ['0-1-1'] * 1 + ['1-1-1'] * 8 + ['1-1'] * 1 + ['0-0-1'] * 1

def rand_res(n):
    res = []
    for idx in range(n):
        sco = random.choice(sco_list)
        res = res + [sco]
        
    return res

["Player " + str(i) for i in range(10)]
rand_res(10)

['1-2', '1-1-1', '1-1-1', '0-1-1', '1-2', '2-1', '2-0', '1-2', '2-0', '0-2']

Notes on behavior:

If a bye is assigned, it's first assigned among subjects with the fewest byes (usually zero) and second among subjects with the fewest points (should be updated to lowest overall ranking).

Remaining matches are distributed by networkx.max_weight_matching with weights derived from each player's place in the rankings.


In [22]:
import random
import polars as pl
import networkx as nx

In [140]:
players = ["Esben",
          "Jeppe",
          "Halfdan",
          "Asger",
          "Bakke"]
players = ["Player " + str(i) for i in range(10)]
if (len(players) > len(set(players))):
    raise ValueError('There are non-unique players.')
elif ("Bye" in players):
    raise ValueError('You have listed "Bye" as a player. Please don\'t.')

players_df = pl.DataFrame({"player": players, "match_pts": 0, "game_pts": 0, "match_tot": 0, "game_tot": 0, "opponents": "", "dropped": False, "mw": 0., "gw": 0., "omw": 0., "ogw": 0., "grp_idx": 0, "pd_count": 0})
players_df

player,match_pts,game_pts,match_tot,game_tot,opponents,dropped,mw,gw,omw,ogw,grp_idx,pd_count
str,i64,i64,i64,i64,str,bool,f64,f64,f64,f64,i64,i64
"""Player 0""",0,0,0,0,"""""",false,0.0,0.0,0.0,0.0,0,0
"""Player 1""",0,0,0,0,"""""",false,0.0,0.0,0.0,0.0,0,0
"""Player 2""",0,0,0,0,"""""",false,0.0,0.0,0.0,0.0,0,0
"""Player 3""",0,0,0,0,"""""",false,0.0,0.0,0.0,0.0,0,0
"""Player 4""",0,0,0,0,"""""",false,0.0,0.0,0.0,0.0,0,0
"""Player 5""",0,0,0,0,"""""",false,0.0,0.0,0.0,0.0,0,0
"""Player 6""",0,0,0,0,"""""",false,0.0,0.0,0.0,0.0,0,0
"""Player 7""",0,0,0,0,"""""",false,0.0,0.0,0.0,0.0,0,0
"""Player 8""",0,0,0,0,"""""",false,0.0,0.0,0.0,0.0,0,0


In [124]:
def pairings(pair_players):
    
    #Remove dropped players from pairing considerations
    pair_players = pair_players.filter(pl.col("dropped") == False)
    
    #If a bye needs to be assigned, do so first.
    has_bye = len(pair_players) % 2 == 1
    
    if (has_bye):
        #Find all players with the lowest number of byes (usually 0) and the highest group index in this group (lowest number of match points, poorest [o](m/g)w)
        bye_players = pair_players.with_columns(
            bye_count = pl.col("opponents").str.count_match("Bye")
        ).filter(
            pl.col("bye_count") == pl.col("bye_count").min()
        ).filter(
            pl.col("grp_idx") == pl.col("grp_idx").max()
        )
        
        bye_pair = bye_players.sample(1).select("player").vstack(
                pl.DataFrame({"player": "Bye"})
            ).with_columns(
                match = 0,
                play_draw = pl.Series(["play", "draw"])
            )
        
        pair_players = pair_players.join(bye_pair, on = "player", how = "anti").with_columns(forbidden = pl.lit(""))
                
    pairs_n = len(pair_players)/2
    
    pairs = pair_players.select("player").with_columns(
        match = 0
    ).clear()
    
    #Extract list of opponents and assign a weight based on rank group
    pair_players = pair_players.with_columns(
        oppo_list = pl.col("opponents").str.split("//"),
        #grp_pts = 1/(1+pl.col("grp_idx")),
        grp_pts = 1 + pl.col("grp_idx").max() - pl.col("grp_idx")
    )
    
    graph_players = dict(zip(pair_players["player"].to_list(), pair_players["oppo_list"].to_list()))
    graph_points = dict(zip(pair_players["player"].to_list(), pair_players["grp_pts"].to_list()))

    bracketGraph = nx.Graph()
    bracketGraph.add_nodes_from(graph_players)
    
    #Sets up possible pairs. The following command becomes slow with > 100 participants. Consider splitting it into piece when groups are big.
    for player in bracketGraph.nodes():
        for opponent in bracketGraph.nodes():
            if opponent not in graph_players[player] and player != opponent:
                #wgt = random.randint(1, 9) #randomizes pairings. Not necessarily desirable.
                wgt = 1 + (graph_points[player] + graph_points[opponent])**2
                bracketGraph.add_edge(player, opponent, weight=wgt)
    
    #Core pairing command. 
    pairings_bg = nx.max_weight_matching(bracketGraph)
    
    #Translate pairings from dict to dataframe and add match number
    for idx, pair_bg in enumerate(pairings_bg):
        pair_df = pl.DataFrame({"player": pair_bg, "match": idx+1}).with_columns(
            match = pl.col("match").cast(pl.Int32),
        )
        pairs = pairs.vstack(pair_df)
        
    #Figure out if the players have been on the play an unequal number of times. If so, give it to whoever has the fewest and if not assign random play/draw.
    pd_pairs = pairs.join(
        pair_players.select("player", "pd_count"), on = "player", how = "left"
    ).groupby("match").agg([pl.col("player"), pl.col("pd_count")]).with_columns(
        play_draw = pl.when(pl.col("pd_count").list.first() > pl.col("pd_count").list.last()).then(pl.lit([["draw", "play"]]))
        .when(pl.col("pd_count").list.first() < pl.col("pd_count").list.last()).then(pl.lit([["play", "draw"]]))
        .otherwise(pl.lit([random.sample(["play", "draw"], 2)]))
    ).drop("pd_count").explode(["player", "play_draw"])
    
    pairs = pairs.join(
        pd_pairs, on = ["player", "match"], how = "left"
    )
    
    if (has_bye):
        pairs = bye_pair.vstack(pairs)
            
    pairs = pairs.pivot(
        values="player", index="match", columns="play_draw", aggregate_function="first"        
    ).select(["match", "play", "draw"])
    
    return pairs

In [141]:
pairings_1 = pairings(players_df)
pairings_1

match,play,draw
i32,str,str
1,"""Player 1""","""Player 8"""
2,"""Player 3""","""Player 6"""
3,"""Player 2""","""Player 7"""
4,"""Player 0""","""Player 9"""
5,"""Player 4""","""Player 5"""


In [175]:
def results(players_df, pairings_df, result_list = None, drop_list = None):
    
    #TODO: Accept manual results
    if (result_list is None):
        result_list = input("How did it go? ")
        
    if (len(pairings_df.filter(pl.col("play") == "Bye")) > 0):
        result_list = ["0-2"] + result_list
    elif (len(pairings_df.filter(pl.col("draw") == "Bye")) > 0):
        result_list = ["2-0"] + result_list
    
    #Combine results_list with pairings_df to create a dataframe with results from each match and assign match points
    result_df = pl.DataFrame({"result": result_list}).with_columns(
        wins = pl.col("result").str.split_exact("-", 2)
    ).unnest("wins").rename(
        {"field_0": "play_wins", "field_1": "draw_wins", "field_2": "games_drawn"}
    ).with_columns(
        play_wins = pl.col("play_wins").cast(pl.Int32),
        draw_wins = pl.col("draw_wins").cast(pl.Int32),
        games_drawn = pl.col("games_drawn").cast(pl.Int32),
        play_points = pl.when(pl.col("play_wins") > pl.col("draw_wins")).then(pl.lit(3))\
            .when(pl.col("play_wins") == pl.col("draw_wins")).then(pl.lit(1))\
            .otherwise(pl.lit(0)),
        draw_points = pl.when(pl.col("draw_wins") > pl.col("play_wins")).then(pl.lit(3))\
            .when(pl.col("draw_wins") == pl.col("play_wins")).then(pl.lit(1))\
            .otherwise(pl.lit(0))
    ).fill_null(0).with_columns(
        total_games = pl.col("play_wins") + pl.col("draw_wins") + pl.col("games_drawn")
    )
    
    result_df = pairings_df.hstack(result_df)
    
    #pivot the match dataframe into a player dataframe.
    play_df = result_df.select(pl.col("^play_.*$"), player = "play", new_opponent = "draw", new_draws = "games_drawn", new_games = "total_games")
    draw_df = result_df.select(pl.col("^draw_.*$"), player = "draw", new_opponent = "play", new_draws = "games_drawn", new_games = "total_games")

    play_df.columns = list(map(lambda x: x.replace("play_", "new_"), play_df.columns))
    draw_df.columns = list(map(lambda x: x.replace("draw_", "new_"), draw_df.columns))
    
    update_df = play_df.vstack(draw_df).filter(pl.col("player") != "Bye")
    
    #Pull dropped players out
    dropped_df = players_df.filter(pl.col("dropped")).drop(["gw", "omw", "ogw", "grp_idx"])
    
    #Update the player dataframe based on the match information and exclude dropped players.
    players_df = players_df.filter(~pl.col("dropped")).drop(["gw", "omw", "ogw", "grp_idx"]).join(update_df, on = "player", how = "left").with_columns(
        match_pts = pl.col("match_pts") + pl.col("new_points"),
        game_pts = pl.col("game_pts") + 3*pl.col("new_wins") + pl.col("new_draws"),
        match_tot = pl.col("match_tot") + pl.lit(1),
        game_tot = pl.col("game_tot") + pl.col("new_games"),
        opponents = (pl.col("opponents") + pl.lit("//") + pl.col("new_opponent")).str.replace("^//", "")
    ).select(pl.col("^[^n].*$"))
    
    #Return the dropped players to the dataframe
    players_df = players_df.vstack(dropped_df).with_columns(
        oppo_struct = pl.col("opponents").str.split("//").list.to_struct()
    ).with_columns(
        mw = 100*pl.col("match_pts")/(3*pl.col("match_tot")),
        gw = 100*pl.col("game_pts")/(3*pl.col("game_tot"))
    ).with_columns(
        mw = pl.when(pl.col("mw")>= 33).then(pl.col("mw")).otherwise(33),
        gw = pl.when(pl.col("gw")>= 33).then(pl.col("gw")).otherwise(33)
    )

    #Determine opponent match/game win percentages.
    player_opponent_df = players_df.unnest("oppo_struct").select(pl.col("^field.*$"), "player").melt(
        id_vars = "player", value_vars = None, value_name = "opponent"
    ).drop("variable").join(
        players_df.select(["player", "mw", "gw"]), left_on = "opponent", right_on = "player", how = "inner"
    ).groupby("player").mean().drop("opponent").rename(
        {"mw": "omw", "gw": "ogw"}
    )
    
    #Determine play/draw information for each player
    pd_players_df = pairings_df.drop("match").melt(
        variable_name = "play_draw",
        value_name = "player"
    ).with_columns(
        pd_chg = pl.when(pl.col("play_draw") == "play").then(1).otherwise(0)
    ).drop("play_draw")

    #Join opponent match/game win% and play/draw information onto player dataframe
    players_df = players_df.join(
        player_opponent_df, on = "player", how = "left"
    ).fill_null(33).sort(["match_pts", "omw", "gw", "ogw"], descending = True).drop("oppo_struct").with_columns(
        dropped = pl.when(pl.col("player").is_in(drop_list)).then(pl.lit(True)).otherwise(pl.col("dropped"))
    ).join(
        pd_players_df, on = "player", how = "left"
    ).with_columns(
        pd_count = pl.col("pd_count") + pl.col("pd_chg")
    ).drop("pd_chg")
    
    #Create an index that encapsulates ranking information.
    grp_idx = []
    for idx, df in enumerate(players_df.groupby(["match_pts", "omw", "gw", "ogw"], maintain_order = True)):
        grp_idx = grp_idx + [idx] * len(df[1])
    players_df = players_df.hstack(pl.DataFrame({"grp_idx": grp_idx}))

    return players_df

results(players_df_2, pairings_3, rand_res(4))

player,match_pts,game_pts,match_tot,game_tot,opponents,dropped,mw,pd_count,gw,omw,ogw,grp_idx
str,i64,i64,i64,i64,str,bool,f64,i64,f64,f64,f64,i64
"""Player 2""",9,18,3,8,"""Player 7//Play…",false,100.0,1,75.0,53.703704,50.216049,0
"""Player 9""",6,15,3,8,"""Player 0//Play…",false,66.666667,2,62.5,70.37037,64.68254,1
"""Player 6""",6,15,3,7,"""Player 3//Play…",false,66.666667,1,71.428571,55.555556,48.611111,2
"""Player 8""",6,12,3,8,"""Player 1//Play…",false,66.666667,1,50.0,44.333333,51.47619,3
"""Player 5""",4,13,3,9,"""Player 4//Play…",false,44.444444,2,48.148148,59.259259,57.539683,4
"""Player 0""",4,10,3,7,"""Player 9//Play…",false,44.444444,2,47.619048,48.148148,47.993827,5
"""Player 7""",3,6,2,5,"""Player 2//Play…",true,50.0,null,40.0,66.5,54.0,6
"""Player 3""",3,6,3,6,"""Player 6//Play…",false,33.333333,2,33.333333,55.555556,59.52381,7
"""Player 4""",3,12,3,8,"""Player 5//Play…",false,33.333333,1,50.0,48.037037,43.716049,8


In [144]:
players_df_1 = results(players_df, pairings_1, rand_res(5))
pairings_2 = pairings(players_df_1)
print(pairings_1)
players_df_1

shape: (5, 3)
┌───────┬──────────┬──────────┐
│ match ┆ play     ┆ draw     │
│ ---   ┆ ---      ┆ ---      │
│ i32   ┆ str      ┆ str      │
╞═══════╪══════════╪══════════╡
│ 1     ┆ Player 1 ┆ Player 8 │
│ 2     ┆ Player 3 ┆ Player 6 │
│ 3     ┆ Player 2 ┆ Player 7 │
│ 4     ┆ Player 0 ┆ Player 9 │
│ 5     ┆ Player 4 ┆ Player 5 │
└───────┴──────────┴──────────┘


player,match_pts,game_pts,match_tot,game_tot,opponents,dropped,mw,pd_count,gw,omw,ogw,grp_idx
str,i64,i64,i64,i64,str,bool,f64,i64,f64,f64,f64,i64
"""Player 2""",3,6,1,2,"""Player 7""",false,100.0,1,100.0,33.0,33.0,0
"""Player 6""",3,6,1,2,"""Player 3""",false,100.0,0,100.0,33.0,33.0,0
"""Player 9""",3,6,1,2,"""Player 0""",false,100.0,0,100.0,33.0,33.0,0
"""Player 5""",3,6,1,3,"""Player 4""",false,100.0,0,66.666667,33.0,33.333333,1
"""Player 8""",3,6,1,3,"""Player 1""",false,100.0,0,66.666667,33.0,33.333333,1
"""Player 1""",0,3,1,3,"""Player 8""",false,33.0,1,33.333333,100.0,66.666667,2
"""Player 4""",0,3,1,3,"""Player 5""",false,33.0,1,33.333333,100.0,66.666667,2
"""Player 0""",0,0,1,2,"""Player 9""",false,33.0,1,33.0,100.0,100.0,3
"""Player 3""",0,0,1,2,"""Player 6""",false,33.0,1,33.0,100.0,100.0,3


In [176]:
players_df_2 = results(players_df_1, pairings_2, rand_res(5), ["Player 7"])
pairings_3 = pairings(players_df_2)
print(pairings_2)
players_df_2

shape: (5, 3)
┌───────┬──────────┬──────────┐
│ match ┆ play     ┆ draw     │
│ ---   ┆ ---      ┆ ---      │
│ i32   ┆ str      ┆ str      │
╞═══════╪══════════╪══════════╡
│ 1     ┆ Player 7 ┆ Player 1 │
│ 2     ┆ Player 8 ┆ Player 4 │
│ 3     ┆ Player 5 ┆ Player 2 │
│ 4     ┆ Player 0 ┆ Player 3 │
│ 5     ┆ Player 9 ┆ Player 6 │
└───────┴──────────┴──────────┘


player,match_pts,game_pts,match_tot,game_tot,opponents,dropped,mw,pd_count,gw,omw,ogw,grp_idx
str,i64,i64,i64,i64,str,bool,f64,i64,f64,f64,f64,i64
"""Player 2""",6,12,2,5,"""Player 7//Play…",false,100.0,1,80.0,50.0,45.0,0
"""Player 6""",6,12,2,5,"""Player 3//Play…",false,100.0,0,80.0,41.5,46.5,1
"""Player 8""",6,12,2,5,"""Player 1//Play…",false,100.0,1,80.0,33.0,33.166667,2
"""Player 9""",3,9,2,5,"""Player 0//Play…",false,50.0,1,60.0,75.0,65.0,3
"""Player 5""",3,9,2,6,"""Player 4//Play…",false,50.0,1,50.0,66.5,56.5,4
"""Player 7""",3,6,2,5,"""Player 2//Play…",true,50.0,1,40.0,66.5,56.666667,5
"""Player 0""",3,6,2,4,"""Player 9//Play…",false,50.0,2,50.0,41.5,46.5,6
"""Player 1""",0,6,2,6,"""Player 8//Play…",false,33.0,1,33.333333,75.0,60.0,7
"""Player 4""",0,3,2,5,"""Player 5//Play…",false,33.0,1,33.0,75.0,65.0,8


In [179]:
players_df_3 = results(players_df_2, pairings_3, rand_res(4), ["Player 4", "Player 1"])
pairings_4 = pairings(players_df_3)
players_df_3

player,match_pts,game_pts,match_tot,game_tot,opponents,dropped,mw,pd_count,gw,omw,ogw,grp_idx
str,i64,i64,i64,i64,str,bool,f64,i64,f64,f64,f64,i64
"""Player 2""",9,18,3,8,"""Player 7//Play…",false,100.0,1,75.0,50.0,48.981481,0
"""Player 9""",6,15,3,7,"""Player 0//Play…",false,66.666667,2,71.428571,66.666667,58.928571,1
"""Player 6""",6,15,3,8,"""Player 3//Play…",false,66.666667,1,62.5,66.666667,59.920635,2
"""Player 0""",6,12,3,7,"""Player 9//Play…",false,66.666667,2,57.142857,44.444444,49.73545,3
"""Player 8""",6,12,3,7,"""Player 1//Play…",false,66.666667,1,57.142857,44.333333,49.095238,4
"""Player 5""",3,12,3,9,"""Player 4//Play…",false,33.333333,2,44.444444,66.666667,58.333333,5
"""Player 3""",3,6,3,6,"""Player 6//Play…",false,33.333333,2,33.333333,66.666667,59.821429,6
"""Player 7""",3,6,2,5,"""Player 2//Play…",true,50.0,null,40.0,66.5,54.0,7
"""Player 4""",3,9,3,7,"""Player 5//Play…",true,33.333333,2,42.857143,44.333333,44.862434,8


In [180]:
results(players_df_3, pairings_4, rand_res(3), ["Player 4", "Player 1"])

player,match_pts,game_pts,match_tot,game_tot,opponents,dropped,mw,pd_count,gw,omw,ogw,grp_idx
str,i64,i64,i64,i64,str,bool,f64,i64,f64,f64,f64,i64
"""Player 9""",9,21,4,10,"""Player 0//Play…",false,75.0,2,70.0,68.75,60.075758,0
"""Player 2""",9,21,4,11,"""Player 7//Play…",false,75.0,2,63.636364,56.25,53.636364,1
"""Player 0""",9,18,4,9,"""Player 9//Play…",false,75.0,2,66.666667,52.0,51.969697,2
"""Player 8""",9,18,4,10,"""Player 1//Play…",false,75.0,2,60.0,43.583333,44.797619,3
"""Player 6""",6,15,4,10,"""Player 3//Play…",false,50.0,2,50.0,64.5,58.409091,4
"""Player 5""",6,18,4,11,"""Player 4//Play…",false,50.0,3,54.545455,61.111111,57.720058,5
"""Player 3""",3,9,4,9,"""Player 6//Play…",false,33.0,2,33.333333,66.666667,58.888889,6
"""Player 7""",3,6,2,5,"""Player 2//Play…",true,50.0,null,40.0,54.0,48.318182,7
"""Player 4""",3,9,3,7,"""Player 5//Play…",true,33.333333,null,42.857143,52.666667,49.181818,8
